<a href="https://colab.research.google.com/github/AprendaPracticando/AnaliticaPythonR1/blob/main/LABS/LAB_14_01_Muestreo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LAB 14.01: Muestra aleatoria simple y muestra estratificada**

En este Lab se utilizarán todas las técnicas aprendidas para realizar el muestreo aleatorio simple y el muestreo estratificado, sobre los datos de la cámara de bienes raíces del condado King County, en Washington; se considerarán las operaciones de venta de los años 2022 y 2023 de los agentes inmobiliarios GINA JEANNOT, FRANK PAINTER – NEOHOMES, y SKYLINE PROPERTIES, que ya han sido limpiados y tratados en otros ejercicios. En otras palabras, ya disponemos de los datos limpios, sin atípicos y ausentes.

Las tareas por realizar son:
1.	Recuperar los datos previamente limpiados y tratados.
2.	Determinar el tamaño de la muestra estadística.
3.	Determinar la muestra aleatoria simple.
4.	Determinar si la muestra es representativa.
5.	Generación de la muestra estratificada.
6.	Verificar que hubo mejora en la representatividad de los estratos.


## **Recuperar los datos previamente limpiados y tratados**

In [ ]:
import pandas as pd

# Se almacena en una variable la liga de acceso a los 
# datos master que se encuentran en Github
url_master='https://raw.githubusercontent.com/AprendaPracticando/AnaliticaPythonR1/main/data/operaciones_master.csv'

# Se enumeran los tipos de datos correctos para los datos.
tipos_esperados={
  'id_operacion':str,
  'fecha_operacion':object,
  'precio':float,
  'public_id':str,
  'codigo_frente_agua':str,
  'codigo_vista':str,
  'codigo_condicion':str,
  'agente_id':str,
  'codigo_pais':str,
  'año':str,
  'año_mes':str,
  'mes':str,
  'vista':str,
  'condición':str,
  'país':str,
  'agente':str,
  'anio_construc':str,
  'latitud':str,
  'longitud':str,
  'codigo_postal':str,
  'm2_construccion':float,
  'm2_terreno':float,
  'm2_sobre_calle':float,
  'm2_sotano':float,
  'pisos':float,
  'recamaras':float,
  'banios':float,
  'anio_renovacion':str
}

# Se leen los datos y se cargan en un DataFrame
operaciones_master=pd.read_csv(url_master, dtype=tipos_esperados)

# La columna fecha_operacion se recuperó como 
# object, y se transforma a datetime
operaciones_master['fecha_operacion']=pd.to_datetime(operaciones_master['fecha_operacion'])

# Se muestra el resultado
operaciones_master.shape


## **Determinar el tamaño de la muestra estadística**

Como podemos comprobar, tenemos 6891 filas, con 28 registros.
Suponiendo que queremos un 99% de confianza con un margen de error del 5%, el tamaño de la muestra se calcularía así.


In [ ]:
# Declara las variables, cuidando que sean del tipo correcto.
N=len(operaciones_master)
p=0.50
q=1-p
E=0.05
Z=2.576

# Se codifica la fórmula, para calcular el tamaño de la 
# muestra (n), y muestra el resultado.
# Toma en cuenta la propiedad conmutativa 'Cuando 
# multiplicamos, el orden de los factores no afecta 
# al producto'.
n=int((Z**2*p*q*N)/((N*E**2)+(Z**2*p*q)))

print(f'El tamaño de la muestra es {n}')


## **Determinar la muestra aleatoria simple**

In [ ]:
# Se calcula la muestra aleatoria simple, considerando
# un tamaño de la muestra de 605.
muestra_aleatoria_simple=operaciones_master.sample(n=605)


## **Determinar si la muestra es representativa**

Vamos a suponer que deseamos que la muestra esté estratificada por el tipo de condición en que se encuentra la propiedad, es decir, por la columna **`condición`**.

Dado que la muestra estratificada pretende reproducir la representatividad que cada estrato tiene, lo primero que debemos determinar es precisamente cuál es la representatividad por reproducir. 

Esto lo logramos conociendo la frecuencia relativa que tiene la característica de estratificación en la población. No hay más representatividad más precisa que esa.


In [ ]:
# Se define cuál es la columna de referencia para
# la estratificación. 
columna='condición'

# Se muestra la tabla de frecuencias absolutas y relativas.
# Se utiliza sort_index para ordenar por el índice.
original=operaciones_master[columna].value_counts()

for condicion, fi in original.sort_index().items():
  hi=fi/len(operaciones_master)
  print(f"{condicion:30s} {fi:10d} {hi:0.4%}")


La Serie pandas llamada **`original`** contiene las frecuencias absolutas para la característica de estratificación, a partir de la población.

Posteriormente, determinamos las frecuencias absolutas y relativas para la característica de estratificación, a partir de la muestra aleatoria simple que ya hemos generado. 


In [ ]:
muestra=muestra_aleatoria_simple[columna].value_counts()

for condicion, fi in muestra.sort_index().items():
  hi=fi/len(muestra_aleatoria_simple)
  print(f"{condicion:30s} {fi:10d} {hi:0.4%}")


La Serie pandas llamada **`muestra`** contiene ahora las frecuencias absolutas para la característica de estratificación, a partir de la muestra.

En este caso, podemos darnos cuenta de que hay discrepancias, pero no son muy significativas; desde luego, habrá casos en donde puede presentarse un mayor problema.


## **Generación de la muestra estratificada**

Si queremos estratificar, de tal manera que la representatividad en una población y en una muestra sean lo más pequeñas posibles, podemos generar muestras aleatorias sobre los estratos, en la proporción que les corresponde a cada uno.

In [ ]:
# Librerías requeridas para procesamiento
from pandas.core.internals import concat
import math

In [ ]:
# Valores generales

# Se especifica la característica de estratificación.
columna='condición'

# Toma en cuenta que hi, la frecuencia relativa, 
# representa la proporción de cada condición
# en la población.
original=operaciones_master[columna].value_counts()
filas=len(operaciones_master)
tamaño_muestra=605

In [ ]:
# Se genera un DataFrame con la estructura del DataFrame
# base. Se genera vacío, con la finalidad de irle agregando
# las filas producto del muestreo.
muestra_estratificada=pd.DataFrame(columns=operaciones_master.columns)

# Ver estructura
muestra_estratificada

In [ ]:
# Se lee de forma secuencial la tabla de frecuencias,
# para determinar cuántas observaciones se deben
# tomar de cada estrato.
for categoria, fi in original.items():
  hi=fi/filas
  # Generamos un conjunto de datos que solo contenga
  # las filas del segmento.
  segmento=operaciones_master[(operaciones_master[columna]==categoria)]

  # Se genera una muestra aleatoria simple, sobre
  # las filas correspondientes al estrato, usando
  # la frecuencia relativa (hi) como proporción.
  elementos_representativos=math.ceil(tamaño_muestra*hi)
  muestra_estrato=segmento.sample(n=elementos_representativos)

  # Se agregan las filas seleccionadas a la muestra
  # estratificada. Se irán acumulando las filas
  # representativas de cada estrato.
  muestra_estratificada=pd.concat([muestra_estratificada,muestra_estrato],ignore_index=True)
  
  # Se muestran las filas que cumplen con el criterio,
  # las filas que constituyen la muestra del segmento,
  # las filas que se acumularán a la muestra estratificada,
  # y las filas acumuladas de la muestra estratificada.
  # Aquí se aprecia cómo va creciendo la muestra.
  print(filas, len(segmento), len(muestra_estrato),len(muestra_estratificada))

# Se muestra la forma final que tiene la muestra estratificada.
print(muestra_estratificada.shape)


In [ ]:
# Se genera la tabla de frecuencias con la muestra
# estratificada, para comprobar que se acerca más
# a la representatividad deseada.
# Es muy poco probable que coincidan exactamente,
# debido a que las muestras aleatorias seleccionan
# diferentes filas cada vez, y el tamaño de observaciones
# a tomar por segmento se redondean.
muestra_est=muestra_estratificada[columna].value_counts()

for condicion, fi in muestra_est.sort_index().items():
  hi=fi/len(muestra_aleatoria_simple)
  print(f"{condicion:30s} {fi:10d} {hi:0.4%}")


## **Verificar que hubo mejora en representatividad de los estratos**

Ahora verificaremos visualmente cuál de las dos muestras, aleatoria simple, o estratificada, representa más acertadamente a los estratos en la población.

Generaremos 3 gráficas de sectores, a partir de Series pandas que contienen las tablas de frecuencias correspondiente a la característica de estratificación, para la población (**`original`**), para la muestra aleatoria simple (**`muestra`**) y para la muestra estratificada (**`muestra_est`**).


In [ ]:
# Librería para poder graficar.
import matplotlib.pyplot as plt

# Creamos la gráfica de sectores, tomando la serie
# original, que contiene la tabla de frecuencias de
# la característica de estratificación, a partir de
# la población.
plt.pie(original, labels=original.index, autopct='%1.2f%%')
plt.title('Población')

# Se genera una nueva gráfica.
plt.figure()
# Creamos la gráfica de sectores, tomando la serie
# muestra, que contiene la tabla de frecuencias de
# la característica de estratificación, a partir de
# la muestra aleatoria simple.
plt.pie(muestra, labels=muestra.index, autopct='%1.2f%%')
plt.title('Muestra aleatoria simple')

# Se genera una nueva gráfica.
plt.figure()
# Creamos la gráfica de sectores, tomando la serie
# muestra, que contiene la tabla de frecuencias de
# la característica de estratificación, a partir de
# la muestra aleatoria simple.
plt.pie(muestra_est, labels=muestra_est.index, autopct='%1.2f%%')
plt.title('Muestra estratificada')

# Se muestran los gráficos
plt.show()
